# ADVANCED SQL 1:  Window Functions & Percentiles

Window Functions is one of the most powerful concepts in SQL data analysis. The Window Function allows us to compare one row to another without doing any joins. This can allow us do simple things like create a running-total as well as tricky things like determine if one row was greater than the previous row and classify it based on our findings.

We connect to MySQL server and workbench and make analysis with the parch-and-posey database. This course is the practicals of the course SQL for Data Analysis at Udacity.

In [1]:
# we import some required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import time
print('Done!')

Done!


In [2]:
import mysql
from mysql.connector import Error
from getpass import getpass

db_name = 'parch_and_posey'
try:
    connection = mysql.connector.connect(host='localhost',
                                         database=db_name,
                                         user=input('Enter UserName:'),
                                         password=getpass('Enter Password:'))
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)

Enter UserName:root
Enter Password:········
Connected to MySQL Server version  8.0.24
You're connected to database:  ('parch_and_posey',)


In [3]:
def query_to_df(query):
    st = time.time()
    # Assert Every Query ends with a semi-colon
    try:
        assert query.endswith(';')
    except AssertionError:
        return 'ERROR: Query Must End with ;'

    # so we never have more than 20 rows displayed
    pd.set_option('display.max_rows', 30) 
    df = None

    # Process the query
    cursor.execute(query)
    columns = cursor.description
    result = []
    for value in cursor.fetchall():
        tmp = {}
        for (index,column) in enumerate(value):
            tmp[columns[index][0]] = [column]
        result.append(tmp)

    # Create a DataFrame from all results
    for ind, data in enumerate(result):
        if ind >= 1:
            x = pd.DataFrame(data)
            df = pd.concat([df, x], ignore_index=True)
        else:
            df = pd.DataFrame(data)
    print(f'Query ran for {time.time()-st} secs!')
    return df

In [4]:
# 1. For the accounts table

query = 'SELECT * FROM accounts LIMIT 3;'
query_to_df(query)

Query ran for 0.2544114589691162 secs!


,id,name,website,lat,longs,primary_poc,sales_rep_id
0,1001,Walmart,www.walmart.com,40.23849561,-75.10329704,Tamara Tuma,321500
1,1011,Exxon Mobil,www.exxonmobil.com,41.16915630,-73.84937379,Sung Shields,321510
2,1021,Apple,www.apple.com,42.29049481,-76.08400942,Jodee Lupo,321520


In [5]:
# 2. For the orders table

query = 'SELECT * FROM orders LIMIT 3;'
query_to_df(query)

Query ran for 0.09993767738342285 secs!


,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1,1001,2015-10-06 17:31:14,123,22,24,169,613.77,164.78,194.88,973.43
1,2,1001,2015-11-05 03:34:33,190,41,57,288,948.10,307.09,462.84,1718.03
2,3,1001,2015-12-04 04:21:55,85,47,0,132,424.15,352.03,0.00,776.18


In [6]:
# 3. For the sales_reps table

query = 'SELECT * FROM sales_reps LIMIT 3;'
query_to_df(query)

Query ran for 0.010325193405151367 secs!


,id,name,region_id
0,321500,Samuel Racine,1
1,321510,Eugena Esser,1
2,321520,Michel Averette,1


In [7]:
# 4. For the web_events table

query = 'SELECT * FROM web_events LIMIT 3;'
query_to_df(query)

Query ran for 0.012826204299926758 secs!


,id,account_id,occurred_at,channel
0,1,1001,2015-10-06 17:13:58,direct
1,2,1001,2015-11-05 03:08:26,direct
2,3,1001,2015-12-04 03:57:24,direct


In [8]:
# 5. For the region table

query = 'SELECT * FROM region LIMIT 3;'
query_to_df(query)

Query ran for 0.007902383804321289 secs!


,id,name
0,1,Northeast
1,2,Midwest
2,3,Southeast


**Let's find the running total of standard_qty papers purchased over time using Window Function**

In [9]:
query_to_df(
"SELECT standard_qty, SUM(standard_qty) OVER (ORDER BY occurred_at) as running_total FROM orders;"
)

Query ran for 14.018959522247314 secs!


,standard_qty,running_total
0,0,0
1,490,490
2,528,1018
3,0,1018
4,492,1510
...,...,...
6907,0,1937478
6908,497,1937975
6909,38,1938013
6910,291,1938304


A **window function** performs an aggregate-like operation on a set of query rows. However, whereas an aggregate operation groups query rows into a single result row, a window function produces a result for each query row:

* The row for which function evaluation occurs is called the current row.
* The query rows related to the current row over which function evaluation occurs comprise the window for the current row.
* The first part of the last query above does a simple `SUM` aggregation. 
* Adding `OVER` designates it as a Window Function
* So everything basically says: Take the sum of standard_qty across all rows leading up to a given row in order by occurred_at.
* window function performs a calculation across a set of table rows that are somehow related to the current row. This is comparable to the type of calculation that can be done with an aggregate function. But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row — the rows retain their separate identities. Behind the scenes, the window function is able to access more than just the current row of the query result.

## OVER
## PARTITION BY

**OVER** and **PARTITION BY**. These are key to window functions. Not every window function uses `PARTITION BY`; we can also use `ORDER BY` or no statement at all depending on the query we want to run.

* The `OVER` clause has three components: `partitioning`, `ordering`, and `framing`. Partitioning is always supported, but support for ordering and framing depends on which type of window function you are using.
* ORDER BY in the OVER clause is not supported for calculating subtotals, for example. You don’t need the data sorted to calculate a sum. Note that the ORDER BY within the OVER clause has nothing to do with an ORDER BY clause found in the query itself.

#### NOTE:

We can’t use window functions and standard aggregations in the same query. More specifically, you can’t include window functions in a GROUP BY clause.

### Understanding Window Functions

In [10]:
query_to_df(
"SELECT account_id, total FROM orders ORDER BY 1, 2;"
)

Query ran for 9.14811372756958 secs!


,account_id,total
0,1001,129
1,1001,132
2,1001,137
3,1001,148
4,1001,158
...,...,...
6907,4501,211
6908,4501,215
6909,4501,224
6910,4501,263


In [11]:
query_to_df(
"SELECT SUM(total) AS total_qty FROM orders;"
)

Query ran for 0.008898019790649414 secs!


,total_qty
0,3675765


In [12]:
query_to_df(
"SELECT account_id, SUM(total) AS total_qty FROM orders GROUP BY 1 ORDER BY 1;"
)

Query ran for 0.6890902519226074 secs!


,account_id,total_qty
0,1001,18924
1,1011,541
2,1021,3810
3,1031,1363
4,1041,2252
...,...,...
345,4461,31887
346,4471,515
347,4481,1380
348,4491,16806


By contrast, window operations do not collapse groups of query rows to a single output row. Instead, they produce a result for each row. Like the preceding queries, the following query uses SUM(), but this time as a window function:

In [13]:
query_to_df(
"SELECT account_id, total, SUM(total) OVER() AS total_std_qty, SUM(total) \
OVER(PARTITION BY account_id) AS accts_total_split FROM orders ORDER BY 1, 2;"
)

Query ran for 16.553161144256592 secs!


,account_id,total,total_std_qty,accts_total_split
0,1001,129,3675765,18924
1,1001,132,3675765,18924
2,1001,137,3675765,18924
3,1001,148,3675765,18924
4,1001,158,3675765,18924
...,...,...,...,...
6907,4501,211,3675765,2244
6908,4501,215,3675765,2244
6909,4501,224,3675765,2244
6910,4501,263,3675765,2244


Each window operation in the query is signified by inclusion of an `OVER` clause that specifies how to partition query rows into groups for processing by the window function:

* The first `OVER` clause is empty, which treats the entire set of query rows as a single partition. The window function thus produces a global sum, but does so for each row.

* The second `OVER` clause partitions rows by account_id, producing a sum per partition (per account_id). The function produces this sum for each partition row.

Window functions are permitted only in the select list and ORDER BY clause. Query result rows are determined from the FROM clause, after WHERE, GROUP BY, and HAVING processing, and windowing execution occurs before ORDER BY, LIMIT, and SELECT DISTINCT.

**NOTE:**

The OVER clause is permitted for many aggregate functions, which therefore can be used as window or nonwindow functions, depending on whether the OVER clause is present or absent:

```
AVG()
BIT_AND()
BIT_OR()
BIT_XOR()
COUNT()
JSON_ARRAYAGG()
JSON_OBJECTAGG()
MAX()
MIN()
STDDEV_POP(), STDDEV(), STD()
STDDEV_SAMP()
SUM()
VAR_POP(), VARIANCE()
VAR_SAMP()
```

MySQL also supports nonaggregate functions that are used only as window functions. For these, the OVER clause is mandatory:

```
CUME_DIST()
DENSE_RANK()
FIRST_VALUE()
LAG()
LAST_VALUE()
LEAD()
NTH_VALUE()
NTILE()
PERCENT_RANK()
RANK()
ROW_NUMBER()
```

As an example of one of those nonaggregate window functions, this query below uses `ROW_NUMBER()`, which produces the row number of each row within its partition. In this case, rows are numbered per `account_id`. By default, partition rows are unordered and row numbering is nondeterministic. To sort partition rows, include an `ORDER BY` clause within the window definition. The query uses unordered and ordered partitions (the row_num1 and row_num2 columns) to illustrate the difference between omitting and including `ORDER BY`:

In [14]:
query_to_df(
"SELECT account_id, total, ROW_NUMBER() OVER(PARTITION BY account_id) AS row_num1, \
ROW_NUMBER() OVER(PARTITION BY account_id ORDER BY total) AS row_num2 FROM orders;"
)

Query ran for 10.760205268859863 secs!


,account_id,total,row_num1,row_num2
0,1001,129,9,1
1,1001,132,3,2
2,1001,137,11,3
3,1001,148,10,4
4,1001,158,13,5
...,...,...,...,...
6907,4501,211,12,9
6908,4501,215,4,10
6909,4501,224,5,11
6910,4501,263,13,12


**Creating a Running Total Using Window Functions**

Create a running total. This time, create a running total of standard_amt_usd (in the orders table) over order time with no date truncation. Your final table should have two columns: one with the amount being added for each new row, and a second with the running total.

In [15]:
query_to_df(
"SELECT standard_amt_usd, SUM(standard_amt_usd) OVER(ORDER BY occurred_at) AS running_total FROM orders;"
)

Query ran for 11.620011329650879 secs!


,standard_amt_usd,running_total
0,0.00,0.00
1,2445.10,2445.10
2,2634.72,5079.82
3,0.00,5079.82
4,2455.08,7534.90
...,...,...
6907,0.00,9668015.22
6908,2480.03,9670495.25
6909,189.62,9670684.87
6910,1452.09,9672136.96


**Creating a Partitioned Running Total Using Window Functions**

Now, modify your query from the previous quiz to include partitions. Still create a running total of standard_amt_usd (in the orders table) over order time, but this time, date truncate occurred_at by year and partition by that same year-truncated occurred_at variable. Your final table should have three columns: One with the amount being added for each row, one for the truncated date, and a final column with the running total within each year.

In [16]:
query_to_df(
"SELECT standard_amt_usd, DATE(occurred_at) year, SUM(standard_amt_usd) \
OVER(PARTITION BY DATE(occurred_at) ORDER BY occurred_at) AS running_total FROM orders;"
)

Query ran for 13.161098003387451 secs!


,standard_amt_usd,year,running_total
0,0.00,2013-12-04,0.00
1,2445.10,2013-12-04,2445.10
2,2634.72,2013-12-04,5079.82
3,0.00,2013-12-05,0.00
4,2455.08,2013-12-05,2455.08
...,...,...,...
6907,0.00,2017-01-01,22040.83
6908,2480.03,2017-01-01,24520.86
6909,189.62,2017-01-01,24710.48
6910,1452.09,2017-01-01,26162.57


### ROW_NUMBER() and RANK() and DENSE_RANK():

* `ROW_NUMBER()` does just what it sounds like. It displays the number of a given row, but within the Window we define. It starts at 1 and numbers the rows according to the `ORDER BY` part of the Windows statement.


* `ROW_NUMBER()` does not require a specified variable within its parenthesis.


* Using the `PARTITION BY` clause within the Window function, we can start the Row-number count at 1 again in each partition.


* `RANK()` might perform the same as `ROW_NUMBER()` but the subtle difference is that if 2 rows or more have the same value for the `ORDER bY` column in the Window statement, they are given the same rank. Whereas `ROW_NUMBER()` would have given them different numbers. The RANK() Clause then skips some values to make up for the numbers.


* `RANK() and DENSE_RANK()` do not expect specific variables within their parenthesis


* `DENSE_RANK()` doesn't skip values after assigning several rows with the same rank.


#### Exercise

**Ranking Total Paper Ordered by Account**

Select the id, account_id, and total variable from the orders table, then create a column called total_rank that ranks this total amount of paper ordered (from highest to lowest) for each account using a partition. Your final table should have these four columns.

In [17]:
query_to_df(
"SELECT id, account_id, total, RANK() OVER(PARTITION BY account_id ORDER BY total DESC) AS total_rank FROM orders;"
)

Query ran for 10.758795738220215 secs!


,id,account_id,total,total_rank
0,4308,1001,1410,1
1,4309,1001,1405,2
2,4316,1001,1384,3
3,4317,1001,1347,4
4,4314,1001,1343,5
...,...,...,...,...
6907,4301,4501,164,9
6908,4300,4501,149,10
6909,4306,4501,126,11
6910,4299,4501,26,12


### QUIZ:

**Aggregates in Window Functions**
```
SELECT id,
       account_id,
       standard_qty,
       MONTH(occurred_at) AS month,
       DENSE_RANK() OVER (PARTITION BY account_id ORDER BY MONTH(occurred_at)) AS dense_rank,
       SUM(standard_qty) OVER (PARTITION BY account_id ORDER BY MONTH(occurred_at)) AS sum_std_qty,
       COUNT(standard_qty) OVER (PARTITION BY account_id ORDER BY MONTH(occurred_at)) AS count_std_qty,
       AVG(standard_qty) OVER (PARTITION BY account_id ORDER BY MONTH(occurred_at)) AS avg_std_qty,
       MIN(standard_qty) OVER (PARTITION BY account_id ORDER BY MONTH(occurred_at)) AS min_std_qty,
       MAX(standard_qty) OVER (PARTITION BY account_id ORDER BY MONTH(occurred_at)) AS max_std_qty
FROM orders
```

In [6]:
query_to_df(
"SELECT id, account_id, standard_qty, MONTH(occurred_at) AS month, \
DENSE_RANK() OVER(PARTITION BY account_id ORDER BY MONTH(occurred_at)) AS dens_rank, \
SUM(standard_qty) OVER(PARTITION BY account_id ORDER BY MONTH(occurred_at)) AS sum_std_qty, \
COUNT(standard_qty) OVER (PARTITION BY account_id ORDER BY MONTH(occurred_at)) AS count_std_qty, \
AVG(standard_qty) OVER (PARTITION BY account_id ORDER BY MONTH(occurred_at)) AS avg_std_qty, \
MIN(standard_qty) OVER (PARTITION BY account_id ORDER BY MONTH(occurred_at)) AS min_std_qty, \
MAX(standard_qty) OVER (PARTITION BY account_id ORDER BY MONTH(occurred_at)) AS max_std_qty \
FROM orders LIMIT 30;"
)

Query ran for 0.6040797233581543 secs!


,id,account_id,standard_qty,month,dens_rank,sum_std_qty,count_std_qty,avg_std_qty,min_std_qty,max_std_qty
0,4,1001,144,1,1,710,2,355.0000,144,566
1,4309,1001,566,1,1,710,2,355.0000,144,566
2,5,1001,108,2,2,1291,4,322.7500,108,566
3,4310,1001,473,2,2,1291,4,322.7500,108,566
4,6,1001,103,3,3,1892,6,315.3333,103,566
5,4311,1001,498,3,3,1892,6,315.3333,103,566
6,7,1001,101,4,4,2490,8,311.2500,101,566
7,4312,1001,497,4,4,2490,8,311.2500,101,566
8,8,1001,95,5,5,3690,12,307.5000,91,566
9,9,1001,91,5,5,3690,12,307.5000,91,566


 Now remove ORDER BY clause in each Window Function of the query above. Evaluate your new query, compare it to the previous results

In [7]:
query_to_df(
"SELECT id, account_id, standard_qty, MONTH(occurred_at) AS month, \
DENSE_RANK() OVER(PARTITION BY account_id) AS dens_rank, \
SUM(standard_qty) OVER(PARTITION BY account_id) AS sum_std_qty, \
COUNT(standard_qty) OVER (PARTITION BY account_id) AS count_std_qty, \
AVG(standard_qty) OVER (PARTITION BY account_id) AS avg_std_qty, \
MIN(standard_qty) OVER (PARTITION BY account_id) AS min_std_qty, \
MAX(standard_qty) OVER (PARTITION BY account_id) AS max_std_qty \
FROM orders LIMIT 30;"
)

Query ran for 0.2780163288116455 secs!


,id,account_id,standard_qty,month,dens_rank,sum_std_qty,count_std_qty,avg_std_qty,min_std_qty,max_std_qty
0,1,1001,123,10,1,7896,28,282.0000,85,566
1,2,1001,190,11,1,7896,28,282.0000,85,566
2,3,1001,85,12,1,7896,28,282.0000,85,566
3,4,1001,144,1,1,7896,28,282.0000,85,566
4,5,1001,108,2,1,7896,28,282.0000,85,566
5,6,1001,103,3,1,7896,28,282.0000,85,566
6,7,1001,101,4,1,7896,28,282.0000,85,566
7,8,1001,95,5,1,7896,28,282.0000,85,566
8,9,1001,91,5,1,7896,28,282.0000,85,566
9,10,1001,94,6,1,7896,28,282.0000,85,566


**Answer the following questions**

* What is the value of dense_rank in every row for the following account_id values <1001, 1011, 1021>?

In [20]:
query_to_df(
"WITH \
t1 AS (SELECT id, account_id, standard_qty, MONTH(occurred_at) AS month, \
DENSE_RANK() OVER(PARTITION BY account_id) AS dens_rank, \
SUM(standard_qty) OVER(PARTITION BY account_id) AS sum_std_qty, \
COUNT(standard_qty) OVER (PARTITION BY account_id) AS count_std_qty, \
AVG(standard_qty) OVER (PARTITION BY account_id) AS avg_std_qty, \
MIN(standard_qty) OVER (PARTITION BY account_id) AS min_std_qty, \
MAX(standard_qty) OVER (PARTITION BY account_id) AS max_std_qty FROM orders), \
\
t2 AS (SELECT DISTINCT account_id, dens_rank FROM t1 WHERE account_id IN(1001, 1011, 1021)) \
SELECT * FROM t2;"
)

Query ran for 0.016864299774169922 secs!


,account_id,dens_rank
0,1001,1
1,1011,1
2,1021,1


Nice! That's correct. dense_rank is constant at 1 for all rows for all account_id values, actually since we removed ORDER BY.

* What is the value of sum_std_qty in the first row for the following account_id values <1001, 1011, 1021>?

In [21]:
query_to_df(
"WITH \
t1 AS (SELECT id, account_id, standard_qty, MONTH(occurred_at) AS month, \
DENSE_RANK() OVER(PARTITION BY account_id) AS dens_rank, \
SUM(standard_qty) OVER(PARTITION BY account_id) AS sum_std_qty, \
COUNT(standard_qty) OVER (PARTITION BY account_id) AS count_std_qty, \
AVG(standard_qty) OVER (PARTITION BY account_id) AS avg_std_qty, \
MIN(standard_qty) OVER (PARTITION BY account_id) AS min_std_qty, \
MAX(standard_qty) OVER (PARTITION BY account_id) AS max_std_qty FROM orders), \
\
t2 AS (SELECT DISTINCT account_id, sum_std_qty FROM t1 WHERE account_id IN(1001, 1011, 1021)) \
SELECT * FROM t2;"
)

Query ran for 0.009042501449584961 secs!


,account_id,sum_std_qty
0,1001,7896
1,1011,527
2,1021,3152


Nice! That's correct. If you look closely, `sum_std_qty` is constant as well for all rows for all account_id values.

* **Reflect...**

What is happening when you omit the ORDER BY clause when doing aggregates with window functions? Use the results from the queries above to guide your thoughts then jot these thoughts down in a few sentences

The `ORDER BY` clause is one of two clauses integral to window functions. The `ORDER` and `PARTITION` define what is referred to as the `“window”`—the ordered subset of data over which calculations are made. Removing `ORDER BY` just leaves an unordered partition; in our query's case, each column's value is simply an aggregation (e.g., sum, count, average, minimum, or maximum) of all the standard_qty values in its respective account_id.

Thus removing the `ORDER BY` clause could lead to...

1. Ranking treats all rows as belonging to just one partition and assigns a uniform Rank of 1 to every row.
2. ALL rows are lumped together per partition and treated as one, yet reported individually per row 

As stackoverflow user mathguy explains...

**The easiest way to think about this - leaving the `ORDER BY` out is equivalent to "ordering" in a way that all rows in the partition are "equal" to each other. Indeed, you can get the same effect by explicitly adding the `ORDER BY` clause like this: `ORDER BY 0` (or "order by" any constant expression), or even, more emphatically, `ORDER BY NULL`.**

### Aliases For Multiple Window Functions:

If we plan to write several `Window` Functions in the same query, using the same `Window`, we can create an alias for the `Window`.

* We define the alias using the `WINDOW` clause. Which normally goes between the `WHERE` clause and the `GROUP BY` clause.
* If our query has neither of those, we put the `WINDOW` clause after `FROM`.
* This will make the query a lot easier to read, while still giving us consistent results.
* Let's see an example with the last query

In [22]:
# Remember that dense_rank is a MySQL keyword so we name the column dens_rank instead

query_to_df(
"SELECT id, account_id, standard_qty, MONTH(occurred_at) AS month, \
DENSE_RANK() OVER main_window AS dens_rank, \
SUM(standard_qty) OVER main_window AS sum_std_qty, \
COUNT(standard_qty) OVER main_window AS count_std_qty, \
AVG(standard_qty) OVER main_window AS avg_std_qty, \
MIN(standard_qty) OVER main_window AS min_std_qty, \
MAX(standard_qty) OVER main_window AS max_std_qty \
FROM orders \
WINDOW main_window AS (PARTITION BY account_id ORDER BY MONTH(occurred_at));"
)

Query ran for 44.469155073165894 secs!


,id,account_id,standard_qty,month,dens_rank,sum_std_qty,count_std_qty,avg_std_qty,min_std_qty,max_std_qty
0,4,1001,144,1,1,710,2,355.0000,144,566
1,4309,1001,566,1,1,710,2,355.0000,144,566
2,5,1001,108,2,2,1291,4,322.7500,108,566
3,4310,1001,473,2,2,1291,4,322.7500,108,566
4,6,1001,103,3,3,1892,6,315.3333,103,566
...,...,...,...,...,...,...,...,...,...,...
6907,4304,4501,159,10,6,759,9,84.3333,5,180
6908,4305,4501,6,11,7,828,11,75.2727,5,180
6909,6911,4501,63,11,7,828,11,75.2727,5,180
6910,6912,4501,61,12,8,1015,13,78.0769,5,180


### Comparing a Row to a previous Row:

* `LAG` Function: It's purpose is to return the value from a previous row to the current row in the table.
* `LEAD` Function: It's purpose is to return the value from the following row to the current row in the table.

**Scenarios for using LAG and LEAD functions**

You can use LAG and LEAD functions whenever you are trying to compare the values in adjacent rows or rows that are offset by a certain number.

Example 1: You have a sales dataset with the following data and need to compare how the market segments fare against each other on profits earned.
```
Market Segment	Profits earned by each market segment
            A	$550
            B	$500
            C	$670
            D	$730
            E	$982
```
Example 2: You have an inventory dataset and need to compare the number of days elapsed between each subsequent order placed for each item

### **Example of LAG function:**

First the Inner Query...

In [26]:
query_to_df(
"SELECT account_id, SUM(standard_qty) std_sum FROM orders GROUP BY 1;"
)

Query ran for 0.6858575344085693 secs!


,account_id,std_sum
0,1001,7896
1,1011,527
2,1021,3152
3,1031,1148
4,1041,836
...,...,...
345,4461,10697
346,4471,417
347,4481,450
348,4491,6411


Next, we start building the outside query and name the inner query as sub...

In [27]:
query_to_df(
"SELECT * FROM \
(SELECT account_id, SUM(standard_qty) std_sum FROM orders GROUP BY 1) AS sub;"
)

Query ran for 0.6688170433044434 secs!


,account_id,std_sum
0,1001,7896
1,1011,527
2,1021,3152
3,1031,1148
4,1041,836
...,...,...
345,4461,10697
346,4471,417
347,4481,450
348,4491,6411


Next, we add the Window Function `OVER (ORDER BY standard_sum)` in the outer query that will create a result set in ascending order based on the standard_sum column.

In [31]:
# Remember that lag is a MySQL keyword so we name the column lagg instead

query_to_df(
"SELECT *, LAG(std_sum) OVER (ORDER BY std_sum) AS lagg FROM \
(SELECT account_id, SUM(standard_qty) std_sum FROM orders GROUP BY 1) as sub;"
)

Query ran for 0.7249755859375 secs!


,account_id,std_sum,lagg
0,1901,0,None
1,3371,79,0
2,1961,102,79
3,3401,116,102
4,3741,117,116
...,...,...,...
345,1281,21943,21448
346,1181,22099,21943
347,3491,22936,22099
348,2631,23321,22936


Each row’s value in `lagg` column is pulled from the previous row in `std_sum` column. E.g., for account_id 1901, the value in `lagg` will come from the previous row. However, since there is no previous row to pull from, the value in `lagg` for account_id 1901 will be NULL. For account_id 3371, the value in `lagg` will be pulled from the previous row (i.e., account_id 1901), which will be 0. This goes on for each row in the table.

Next, to compare the values between the rows, we need to use both columns (`std_sum` and `lagg`). We add a new column named `lag_difference`, which subtracts the `lagg` value from the value in `std_sum` for each row in the table:<br>
**`std_sum - LAG(std_sum) OVER (ORDER BY std_sum) AS lag_difference`**

In [32]:
# Remember that lag is a MySQL keyword so we name the column lagg instead

query_to_df(
"SELECT *, LAG(std_sum) OVER (ORDER BY std_sum) AS lagg, \
std_sum - LAG(std_sum) OVER (ORDER BY std_sum) AS lag_difference \
FROM \
(SELECT account_id, SUM(standard_qty) std_sum FROM orders GROUP BY 1) as sub;"
)

Query ran for 0.8487730026245117 secs!


,account_id,std_sum,lagg,lag_difference
0,1901,0,None,None
1,3371,79,0,79
2,1961,102,79,23
3,3401,116,102,14
4,3741,117,116,1
...,...,...,...,...
345,1281,21943,21448,495
346,1181,22099,21943,156
347,3491,22936,22099,837
348,2631,23321,22936,385


Each value in `lag_difference` is comparing the row values between the 2 columns (`std_sum` and `lagg`). E.g., since the value for `lagg` in the case of account_id 1901 is NULL, the value in lag_difference for account_id 1901 will be NULL. However, for account_id 3371, the value in lag_difference will compare the value 79 (`std_sum` for account_id 3371) with 0 (`lagg` for account_id 3371) resulting in 79. This goes on for each row in the table.

### **Example of LEAD function:**

The `LEAD` function maintains similar steps, but basically does the opposite of `LAG` as it compares each row to its succeeding row rather than to the preceding row.

First the Inner Query...

In [33]:
query_to_df(
"SELECT account_id, SUM(standard_qty) std_sum FROM orders GROUP BY 1;"
)

Query ran for 0.6771881580352783 secs!


,account_id,std_sum
0,1001,7896
1,1011,527
2,1021,3152
3,1031,1148
4,1041,836
...,...,...
345,4461,10697
346,4471,417
347,4481,450
348,4491,6411


Next, we start building the outside query and name the inner query as sub...

In [34]:
query_to_df(
"SELECT * FROM \
(SELECT account_id, SUM(standard_qty) std_sum FROM orders GROUP BY 1) AS sub;"
)

Query ran for 0.6688277721405029 secs!


,account_id,std_sum
0,1001,7896
1,1011,527
2,1021,3152
3,1031,1148
4,1041,836
...,...,...
345,4461,10697
346,4471,417
347,4481,450
348,4491,6411


Next, we add the Window Function `OVER (ORDER BY standard_sum)` in the outer query that will create a result set in ascending order based on the standard_sum column.

In [37]:
# Remember that lead is a MySQL keyword so we name the column leadd instead

query_to_df(
"SELECT *, LEAD(std_sum) OVER(ORDER BY std_sum) AS leadd FROM \
(SELECT account_id, SUM(standard_qty) std_sum FROM orders GROUP BY 1) AS sub;"
)

Query ran for 0.7814993858337402 secs!


,account_id,std_sum,leadd
0,1901,0,79
1,3371,79,102
2,1961,102,116
3,3401,116,117
4,3741,117,123
...,...,...,...
345,1281,21943,22099
346,1181,22099,22936
347,3491,22936,23321
348,2631,23321,41617


This new column named `leadd` uses the values from `std_sum`. Each row’s value in `leadd` is pulled from the row after it in `std_sum` column. E.g., for account_id 1901, the value in `leadd` will come from the row following it (i.e., for account_id 3371). Since the value is 79, the value in `leadd` for account_id 1901 will be 79. For account_id 3371, the value in `leadd` will be pulled from the following row (i.e., account_id 1961), which will be 102. This goes on for each row in the table.

Next, to compare the values between the rows, we need to use both columns (`std_sum` and `leadd`). We add a new column named `lead_difference`, which subtracts the `std_sum` value from the value in `leadd` for each row in the table:<br>
**`LEAD(std_sum) OVER (ORDER BY std_sum) - std_sum AS lead_difference`**

In [38]:
# Remember that lead is a MySQL keyword so we name the column leadd instead

query_to_df(
"SELECT *, LEAD(std_sum) OVER(ORDER BY std_sum) AS leadd, \
LEAD(std_sum) OVER(ORDER BY std_sum) - std_sum AS lead_difference \
FROM \
(SELECT account_id, SUM(standard_qty) std_sum FROM orders GROUP BY 1) AS sub;"
)

Query ran for 0.7797198295593262 secs!


,account_id,std_sum,leadd,lead_difference
0,1901,0,79,79
1,3371,79,102,23
2,1961,102,116,14
3,3401,116,117,1
4,3741,117,123,6
...,...,...,...,...
345,1281,21943,22099,156
346,1181,22099,22936,837
347,3491,22936,23321,385
348,2631,23321,41617,18296


Each value in `lead_difference` is comparing the row values between the 2 columns (`std_sum` and `leadd`). E.g., for account_id 1901, the value in `lead_difference` will compare the value 0 (`std_sum` for account_id 1901) with 79 (`leadd` for account_id 1901) resulting in 79. This goes on for each row in the table.

#### QUIZ: Comparing a Row to Previous Row

In the previous exercise we outlined how to compare a row to a previous or subsequent row. This technique can be useful when analyzing **time-based** events.<br>**Imagine you're an analyst at Parch & Posey and you want to determine how the current order's total revenue ("total" meaning from sales of all types of paper) compares to the next order's total revenue**.

You'll need to use `occurred_at` and `total_amt_usd` in the orders table along with `LEAD` to do so. In your query results, there should be four columns: `occurred_at`, `total_amt_usd`, `lead`, and `lead_difference`.

In [10]:
query_to_df(
"WITH \
t1 AS (SELECT occurred_at, SUM(total_amt_usd) total_amt_usd FROM orders GROUP BY 1), \
t2 AS (SELECT *, LEAD(total_amt_usd) OVER(ORDER BY occurred_at) AS leadd, \
LEAD(total_amt_usd) OVER(ORDER BY occurred_at) - total_amt_usd AS lead_difference FROM t1) \
SELECT * FROM t2;"
)

Query ran for 19.66036820411682 secs!


,occurred_at,total_amt_usd,leadd,lead_difference
0,2013-12-04 04:22:44,627.48,2646.77,2019.29
1,2013-12-04 04:45:54,2646.77,2709.62,62.85
2,2013-12-04 04:53:25,2709.62,277.13,-2432.49
3,2013-12-05 20:29:16,277.13,3001.85,2724.72
4,2013-12-05 20:33:56,3001.85,2802.90,-198.95
...,...,...,...,...
6903,2017-01-01 21:04:25,892.85,2666.79,1773.94
6904,2017-01-01 22:17:26,2666.79,1854.57,-812.22
6905,2017-01-01 22:29:50,1854.57,1932.85,78.28
6906,2017-01-01 23:50:16,1932.85,6451.76,4518.91


To create a better analysis, let's add a `pct_diff` column that computes the `lead_difference` as a percentage of the `total_amt_usd`. <br>This lets us see how the increase or decrease in the next order compares to the entire value of the previous order.<br>We can easily do this using a With and Case statement.

In [12]:
query_to_df(
"WITH \
t1 AS (SELECT occurred_at, SUM(total_amt_usd) total_amt_usd FROM orders GROUP BY 1), \
t2 AS (SELECT *, LEAD(total_amt_usd) OVER(ORDER BY occurred_at) AS leadd, \
LEAD(total_amt_usd) OVER(ORDER BY occurred_at) - total_amt_usd AS lead_difference FROM t1), \
t3 AS (SELECT *, CASE WHEN lead_difference = 0 OR lead_difference IS NULL THEN 0 \
ELSE (lead_difference/total_amt_usd)*100 END AS pct_difference FROM t2) \
SELECT * FROM t3;"
)

Query ran for 20.58346176147461 secs!


,occurred_at,total_amt_usd,leadd,lead_difference,pct_difference
0,2013-12-04 04:22:44,627.48,2646.77,2019.29,321.809460
1,2013-12-04 04:45:54,2646.77,2709.62,62.85,2.374592
2,2013-12-04 04:53:25,2709.62,277.13,-2432.49,-89.772367
3,2013-12-05 20:29:16,277.13,3001.85,2724.72,983.192004
4,2013-12-05 20:33:56,3001.85,2802.90,-198.95,-6.627580
...,...,...,...,...,...
6903,2017-01-01 21:04:25,892.85,2666.79,1773.94,198.682869
6904,2017-01-01 22:17:26,2666.79,1854.57,-812.22,-30.456841
6905,2017-01-01 22:29:50,1854.57,1932.85,78.28,4.220925
6906,2017-01-01 23:50:16,1932.85,6451.76,4518.91,233.795173


## Percentiles:

Arguably, the best way to analyze sales records is to look at the Percentiles, to see where most sale orders fall. We can do this with the Ntile() Window function.

### **Ntile():**

The `NTILE()` function allows us to see the percentile or any other sub-division that a given row falls into. 
* The number specified into the `NTILE()` function is the number of parts into which we'd divide the window into.
> 100 means **Percentile**<br>
5 means **Quintile**<br>
4 means **Quartile**

* ORDER BY can help us choose which column to use to determine the divisions. 
* Thus if we choose a quartile of 4 divisions, then for each row, the `NTILe(4)` function will look at the value of the `ORDER BY` column in that row and compare it to all the other rows in the `Window` and then print the Quartile that the value falls into. Thus a value of say 0, will fall in the first quartile, while the highest value will fall in the 4th quartile.

**Expert Tip**

In cases with relatively few rows in a window, the NTILE function doesn’t calculate exactly as you might expect. For example, If you only had two records and you were measuring percentiles, you’d expect one record to define the 1st percentile, and the other record to define the 100th percentile. Using the NTILE function, what you’d actually see is one record in the 1st percentile, and one in the 2nd percentile.

In other words, when you use a NTILE function but the number of rows in the partition is less than the NTILE(number of groups), then NTILE will divide the rows into as many groups as there are members (rows) in the set but then stop short of the requested number of groups. If you’re working with very small windows, keep this in mind and consider using quartiles or similarly small bands.

### QUIZ: Percentiles with Partitions

You can use partitions with percentiles to determine the percentile of a specific subset of all rows. Imagine you're an analyst at Parch & Posey and you want to determine the largest orders (in terms of quantity) a specific customer has made to encourage them to order more similarly sized large orders. You only want to consider the NTILE for that customer's account_id.

**write three queries (separately) that reflect each of the following:...**

1. Use the **`NTILE`** functionality to divide the accounts into 4 levels in terms of the amount of standard_qty for their orders. Your resulting table should have the **`account_id`**, the **`occurred_at`** time for each order, the **`total`** amount of standard_qty paper purchased, and one of four levels in a **`standard_quartile`** column.

In [80]:
query_to_df(
"WITH \
t1 AS (SELECT account_id, occurred_at, SUM(standard_qty) total_std_qty FROM orders GROUP BY 1, 2), \
t2 AS (SELECT *, NTILE(4) OVER(PARTITION BY account_id ORDER BY total_std_qty) AS std_quartile FROM t1) \
SELECT * FROM t2;"
)

Query ran for 12.063676595687866 secs!


,account_id,occurred_at,total_std_qty,std_quartile
0,1001,2015-12-04 04:21:55,85,1
1,1001,2016-05-31 21:22:48,91,1
2,1001,2016-06-30 12:32:05,94,1
3,1001,2016-05-01 15:55:51,95,1
4,1001,2016-10-26 20:31:30,97,1
...,...,...,...,...
6903,4501,2016-07-29 20:06:39,111,3
6904,4501,2016-12-21 13:43:26,126,3
6905,4501,2016-09-25 01:44:03,158,4
6906,4501,2016-10-24 08:50:37,159,4


2. Use the **`NTILE`** functionality to divide the accounts into two levels in terms of the amount of gloss_qty for their orders. Your resulting table should have the **`account_id`**, the **`occurred_at`** time for each order, the **`total`** amount of gloss_qty paper purchased, and one of two levels in a **`gloss_half`** column.

In [72]:
query_to_df(
"WITH \
t1 AS (SELECT account_id, occurred_at, SUM(gloss_qty) total_gloss_qty FROM orders GROUP BY 1, 2), \
t2 AS (SELECT *, NTILE(2) OVER(PARTITION BY account_id ORDER BY total_gloss_qty) AS gloss_half FROM t1) \
SELECT * FROM t2;"
)

Query ran for 11.556330680847168 secs!


,account_id,occurred_at,total_gloss_qty,gloss_half
0,1001,2016-09-26 23:28:25,10,1
1,1001,2016-05-31 21:22:48,16,1
2,1001,2015-10-06 17:31:14,22,1
3,1001,2016-03-02 15:29:32,24,1
4,1001,2016-02-01 19:27:27,29,1
...,...,...,...,...
6903,4501,2016-11-22 06:52:22,67,2
6904,4501,2016-07-29 19:58:32,91,2
6905,4501,2016-08-27 00:58:11,94,2
6906,4501,2016-12-21 13:30:42,150,2


3. Use the **`NTILE`** functionality to divide the orders for each account into 100 levels in terms of the amount of total_amt_usd for their orders. Your resulting table should have the **`account_id`**, the **`occurred_at`** time for each order, the **`total`** amount of total_amt_usd paper purchased, and one of 100 levels in a **`total_percentile`** column.

In [74]:
query_to_df(
"WITH \
t1 AS (SELECT account_id, occurred_at, SUM(total_amt_usd) total_amt_usd FROM orders GROUP BY 1, 2), \
t2 AS (SELECT *, NTILE(100) OVER(PARTITION BY account_id ORDER BY total_amt_usd) AS total_percentile FROM t1) \
SELECT * FROM t2 LIMIT 30;"
)

Query ran for 0.11573123931884766 secs!


,account_id,occurred_at,total_amt_usd,total_percentile
0,1001,2016-05-31 21:22:48,752.57,1
1,1001,2016-07-30 03:26:30,773.63,2
2,1001,2015-12-04 04:21:55,776.18,3
3,1001,2016-06-30 12:32:05,878.56,4
4,1001,2016-09-26 23:28:25,951.14,5
5,1001,2016-01-02 01:18:24,958.24,6
6,1001,2015-10-06 17:31:14,973.43,7
7,1001,2016-02-01 19:27:27,983.49,8
8,1001,2016-03-02 15:29:32,1067.25,9
9,1001,2016-08-28 07:13:39,1182.61,10


## More on Window Functions:

* If **`OVER()`** is empty, the window consists of all query rows and the window function computes a result using all rows. Otherwise, the clauses present within the parentheses determine which query rows are used to compute the function result and how they are partitioned and ordered


* **partition_clause**: A **`PARTITION BY`** clause indicates how to divide the query rows into groups. The window function result for a given row is based on the rows of the partition that contains the row. If `PARTITION BY` is omitted, there is a single partition consisting of all query rows.


* Standard SQL requires `PARTITION BY` to be followed by column names only. A MySQL extension is to permit expressions, not just column names. For example, if a table contains a TIMESTAMP column named ts, standard SQL permits `PARTITION BY ts` but not `PARTITION BY HOUR(ts)`, whereas MySQL permits both.


* **order_clause**: An **`ORDER BY`** clause indicates how to sort rows in each partition. Partition rows that are equal according to the `ORDER BY` clause are considered peers. If `ORDER BY` is omitted, partition rows are unordered, with no processing order implied, and all partition rows are peers.


* Each **`ORDER BY`** expression optionally can be followed by `ASC` or `DESC` to indicate sort direction. The default is `ASC` if no direction is specified. `NULL` values sort first for ascending sorts, last for descending sorts.


* An **`ORDER BY`** in a window definition applies within individual partitions. To sort the result set as a whole, include an **`ORDER BY`** at the query top level.


* frame_clause: A **`frame`** is a subset of the current partition and the frame clause specifies how to define the subset. The frame clause has many subclauses of its own **[LINK](https://dev.mysql.com/doc/refman/8.0/en/window-functions-frames.html)**

### Frame Specification for Window Function:

The definition of a window used with a window function can include a frame clause. A **frame** is a subset of the current partition and the **frame clause** specifies how to define the subset.

**Frames** are determined with respect to the current row, which enables a frame to move within a partition depending on the location of the current row within its partition. Examples:

* By defining a frame to be all rows from the partition start to the current row, you can compute running totals for each row.
* By defining a frame as extending N rows on either side of the current row, you can compute rolling averages

The following query demonstrates the use of moving frames to compute running totals within each group of time-ordered level values, as well as rolling averages computed from the current row and the rows that immediately precede and follow it:

In [89]:
query_to_df(
"SELECT account_id, DATE_FORMAT(occurred_at, '%Y-%m-%d %H:%i') time_, gloss_qty, \
SUM(gloss_qty) OVER(PARTITION BY account_id ORDER BY DATE_FORMAT(occurred_at, '%Y-%m-%d %H:%i') \
ROWS UNBOUNDED PRECEDING) AS running_total, \
AVG(gloss_qty) OVER(PARTITION BY account_id ORDER BY DATE_FORMAT(occurred_at, '%Y-%m-%d %H:%i') \
ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) AS running_avg FROM orders;"
)

Query ran for 15.832660436630249 secs!


,account_id,time_,gloss_qty,running_total,running_avg
0,1001,2015-10-06 17:31,22,22,317.0000
1,1001,2015-11-05 03:25,612,634,225.0000
2,1001,2015-11-05 03:34,41,675,416.6667
3,1001,2015-12-04 04:01,597,1272,228.3333
4,1001,2015-12-04 04:21,47,1319,429.6667
...,...,...,...,...,...
6907,4501,2016-10-24 08:50,6,442,24.3333
6908,4501,2016-11-22 06:52,67,509,24.3333
6909,4501,2016-11-22 06:57,0,509,72.3333
6910,4501,2016-12-21 13:30,150,659,50.0000


From the above query, we compute a **running-total** and **running-avg** that is partitioned per `account_id`. This is ordered by time from the earliest order per account_id to the latest order. Thus for the first order of account_id 1001, the running-total starts at 22, for the next order it goes to **`22+612=634`**, for the next order it's **`22+612+41=675`** and so on...

Then we also compute the running average starting at 1 row before to 1 row after current row, both edges inclusive.
Thus for first order of `gloss_qty=22`, since no preceding row, the running average is **`22+612/2=317`**, for the next order of `gloss_qty=612`, the running-average is **`22+612+41/3=225`** and so on...

**NOTE:**

Standard SQL specifies that window functions that operate on the entire partition should have no frame clause. MySQL permits a frame clause for such functions but ignores it. These functions use the entire partition even if a frame is specified:

* CUME_DIST()
* DENSE_RANK()
* LAG()
* LEAD()
* NTILE()
* PERCENT_RANK()
* RANK()
* ROW_NUMBER()

**In the absence of a frame clause, the default frame depends on whether an ORDER BY clause is present.**

## Frame-Units and Frame-Extents

### Frame-Units:

The frame_units value indicates the type of relationship between the current row and frame rows:

* **ROWS:** The frame is defined by beginning and ending row positions. Offsets are differences in row numbers from the current row number.

* **RANGE:** The frame is defined by rows within a value range. Offsets are differences in row values from the current row value.

### Frame-Extents:

The frame_extent value indicates the start and end points of the frame. You can specify just the start of the frame (in which case the current row is implicitly the end) or use `BETWEEN` to specify both frame endpoints:

* With BETWEEN syntax, frame_start must not occur later than frame_end.

The permitted frame_start and frame_end values have these meanings:

* CURRENT ROW: For ROWS, the bound is the current row. For RANGE, the bound is the peers of the current row.

* UNBOUNDED PRECEDING: The bound is the first partition row.

* UNBOUNDED FOLLOWING: The bound is the last partition row.

## FIRST_VALUE(), LAST_VALUE() and NTH_VALUE():

* **FIRST_VALUE()**: Value of argument from first row of window frame
* **LAST_VALUE()**: Value of argument from last row of window frame
* **NTH_VALUE()**: Value of argument from N-th row of window frame

The following query demonstrates FIRST_VALUE(), LAST_VALUE(), and two instances of NTH_VALUE():

In [92]:
query_to_df(
"SELECT account_id, DATE_FORMAT(occurred_at, '%Y-%m-%d %H:%i') time_, total_amt_usd, \
FIRST_VALUE(total_amt_usd) OVER W AS first, \
LAST_VALUE(total_amt_usd) OVER W AS last, \
NTH_VALUE(total_amt_usd, 2) OVER W AS second, \
NTH_VALUE(total_amt_usd, 4) OVER W AS fourth \
FROM orders \
WINDOW W AS (PARTITION BY account_id ORDER BY DATE_FORMAT(occurred_at, '%Y-%m-%d %H:%i') \
ROWS UNBOUNDED PRECEDING);"
)

Query ran for 10.537724494934082 secs!


,account_id,time_,total_amt_usd,first,last,second,fourth
0,1001,2015-10-06 17:31,973.43,973.43,973.43,None,None
1,1001,2015-11-05 03:25,8757.18,973.43,8757.18,8757.18,None
2,1001,2015-11-05 03:34,1718.03,973.43,1718.03,8757.18,None
3,1001,2015-12-04 04:01,9426.71,973.43,9426.71,8757.18,9426.71
4,1001,2015-12-04 04:21,776.18,973.43,776.18,8757.18,9426.71
...,...,...,...,...,...,...,...
6907,4501,2016-10-24 08:50,1122.55,157.24,1122.55,875.54,1486.06
6908,4501,2016-11-22 06:52,1473.92,157.24,1473.92,875.54,1486.06
6909,4501,2016-11-22 06:57,86.78,157.24,86.78,875.54,1486.06
6910,4501,2016-12-21 13:30,1850.13,157.24,1850.13,875.54,1486.06


In [13]:
# Change False to True below and run cell to terminate connection

if True and connection.is_connected():
    cursor.close()
    connection.close()
    print(f'Connection Terminated: {record} Database.')

Connection Terminated: ('parch_and_posey',) Database.
